# Normalization, HVG Selection, and Dimensionality Reduction

## Overview
This notebook performs normalization, highly variable gene (HVG) selection, and dimensionality reduction on QC-filtered scRNA-seq data.

### Objectives
1. Normalize counts (library size normalization + log transformation)
2. Identify highly variable genes
3. Scale data for PCA
4. Perform PCA and UMAP
5. Initial clustering for QC assessment

### Key Concepts
- **Normalization**: Correct for differences in sequencing depth between cells
- **HVG selection**: Focus on genes with high biological variability
- **PCA**: Reduce dimensionality while preserving variance
- **UMAP**: Non-linear embedding for visualization

---

## 1. Setup

In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import warnings

warnings.filterwarnings('ignore')

# Scanpy settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white')

# Project paths
PROJECT_ROOT = Path("../..").resolve()
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed' / 'scrna'
FIGURES = PROJECT_ROOT / 'results' / 'figures'
CONFIG_PATH = PROJECT_ROOT / 'config' / 'analysis_params.yaml'

# Load configuration
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Set random seed
SEED = config['random_seed']
np.random.seed(SEED)

print(f"Random seed: {SEED}")

In [ ]:
# Display relevant parameters
print("Normalization parameters:")
print(f"  Target sum: {config['normalization']['target_sum']}")
print(f"  Log transform: {config['normalization']['log_transform']}")

print("\nFeature selection parameters:")
print(f"  N top genes: {config['feature_selection']['n_top_genes']}")
print(f"  Flavor: {config['feature_selection']['flavor']}")

print("\nDimensionality reduction parameters:")
print(f"  N PCs: {config['dim_reduction']['n_pcs']}")
print(f"  N neighbors: {config['dim_reduction']['n_neighbors']}")

## 2. Load Filtered Data

In [ ]:
# Load QC-filtered data
geo_id = "GSE115978"  # Modify for each dataset
input_path = DATA_PROCESSED / f'{geo_id}_filtered.h5ad'

# Check if file exists
if input_path.exists():
    adata = sc.read_h5ad(input_path)
    print(f"Loaded: {input_path}")
    print(f"Cells: {adata.n_obs}")
    print(f"Genes: {adata.n_vars}")
else:
    print(f"File not found: {input_path}")
    print("Please run 02a_qc_filtering.ipynb first")

## 3. Normalization

We use library size normalization followed by log transformation:
1. Normalize each cell to have the same total counts (target_sum)
2. Log transform: log(1 + x)

This is the standard scanpy workflow and is appropriate for most analyses.

In [ ]:
# Store raw counts
adata.layers['counts'] = adata.X.copy()

# Normalize to target sum
sc.pp.normalize_total(
    adata,
    target_sum=config['normalization']['target_sum']
)

# Log transform
if config['normalization']['log_transform']:
    sc.pp.log1p(adata)

# Store normalized counts
adata.layers['normalized'] = adata.X.copy()

print("Normalization complete")
print(f"Stored layers: {list(adata.layers.keys())}")

## 4. Highly Variable Gene Selection

Identify genes with high cell-to-cell variability, which are likely to be biologically informative.

### Methods
- **seurat_v3**: Models mean-variance relationship and selects genes with high residual variance
- **cell_ranger**: Uses dispersion-based selection

For integration across batches, consider using `batch_key` parameter.

In [ ]:
# Identify highly variable genes
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=config['feature_selection']['n_top_genes'],
    flavor=config['feature_selection']['flavor'],
    # batch_key=config['feature_selection']['batch_key'],  # Uncomment for batch-aware HVG
)

n_hvg = adata.var['highly_variable'].sum()
print(f"\nSelected {n_hvg} highly variable genes")

In [ ]:
# Visualize HVG selection
sc.pl.highly_variable_genes(adata, save=f'_{geo_id}_hvg.png')

In [ ]:
# Show top HVGs
hvg_df = adata.var[adata.var['highly_variable']].sort_values('dispersions_norm', ascending=False)
print("\nTop 20 highly variable genes:")
print(hvg_df.head(20)[['means', 'dispersions', 'dispersions_norm']])

## 5. Scaling

Scale each gene to unit variance. This is required for PCA.

**Note**: Scaling is only applied to HVGs to reduce memory usage. The full matrix remains unscaled.

In [ ]:
# Regress out unwanted sources of variation (optional)
# Uncomment if needed:
# sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

# Scale data
sc.pp.scale(adata, max_value=10)

print("Scaling complete")

## 6. Principal Component Analysis (PCA)

Reduce dimensionality while preserving variance. Use only HVGs for PCA.

In [ ]:
# Run PCA
sc.tl.pca(
    adata,
    n_comps=config['dim_reduction']['n_pcs'],
    use_highly_variable=True,
    svd_solver='arpack'
)

print(f"PCA complete: {adata.obsm['X_pca'].shape}")

In [ ]:
# Visualize variance explained
sc.pl.pca_variance_ratio(
    adata,
    n_pcs=50,
    log=True,
    save=f'_{geo_id}_pca_variance.png'
)

In [ ]:
# Determine number of PCs to use
# Look for elbow in variance explained
cumvar = np.cumsum(adata.uns['pca']['variance_ratio'])

# Find PC that captures 90% variance
n_pcs_90 = np.argmax(cumvar >= 0.9) + 1
print(f"\nPCs needed for 90% variance: {n_pcs_90}")

# We'll use the configured number of PCs
n_pcs = config['dim_reduction']['n_pcs']
print(f"Using {n_pcs} PCs for downstream analysis")

In [ ]:
# Visualize PCA
sc.pl.pca(
    adata,
    color=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'],
    ncols=3,
    save=f'_{geo_id}_pca_qc.png'
)

## 7. Neighborhood Graph and UMAP

Build a neighborhood graph in PCA space and compute UMAP embedding.

In [ ]:
# Compute neighborhood graph
sc.pp.neighbors(
    adata,
    n_neighbors=config['dim_reduction']['n_neighbors'],
    n_pcs=n_pcs,
    random_state=SEED
)

print("Neighborhood graph computed")

In [ ]:
# Compute UMAP
sc.tl.umap(adata, random_state=SEED)

print("UMAP computed")

In [ ]:
# Visualize UMAP colored by QC metrics
sc.pl.umap(
    adata,
    color=['total_counts', 'n_genes_by_counts', 'pct_counts_mt', 'pct_counts_ribo'],
    ncols=2,
    save=f'_{geo_id}_umap_qc.png'
)

## 8. Initial Clustering

Perform initial clustering to assess data quality and identify major cell populations.

In [ ]:
# Leiden clustering at multiple resolutions
resolutions = config['clustering']['resolution']

for res in resolutions:
    key = f'leiden_{res}'
    sc.tl.leiden(adata, resolution=res, key_added=key, random_state=SEED)
    n_clusters = adata.obs[key].nunique()
    print(f"Resolution {res}: {n_clusters} clusters")

In [ ]:
# Visualize clustering results
default_res = config['clustering']['default_resolution']
default_key = f'leiden_{default_res}'

sc.pl.umap(
    adata,
    color=[default_key],
    legend_loc='on data',
    title=f'Leiden clustering (res={default_res})',
    save=f'_{geo_id}_umap_clusters.png'
)

In [ ]:
# Compare multiple resolutions
sc.pl.umap(
    adata,
    color=[f'leiden_{r}' for r in [0.3, 0.5, 1.0]],
    ncols=3,
    save=f'_{geo_id}_umap_resolutions.png'
)

## 9. Marker Gene Visualization

Check expression of known marker genes to validate cell populations.

In [ ]:
# Define marker genes from config
markers = config['annotation']['markers']

# Flatten marker dictionary
all_markers = []
for cell_type, genes in markers.items():
    all_markers.extend(genes)

# Filter to genes present in data
present_markers = [g for g in all_markers if g in adata.var_names]
missing_markers = [g for g in all_markers if g not in adata.var_names]

print(f"Markers present: {len(present_markers)}/{len(all_markers)}")
if missing_markers:
    print(f"Missing: {missing_markers[:10]}...")

In [ ]:
# Visualize key markers
key_markers = ['CD3D', 'CD8A', 'CD4', 'FOXP3', 'CD14', 'CD68', 'MS4A1', 'EPCAM', 'PECAM1']
key_markers = [m for m in key_markers if m in adata.var_names]

if key_markers:
    sc.pl.umap(
        adata,
        color=key_markers,
        ncols=3,
        save=f'_{geo_id}_umap_markers.png'
    )

## 10. Save Processed Data

In [ ]:
# Save processed data with all embeddings
output_path = DATA_PROCESSED / f'{geo_id}_processed.h5ad'
adata.write(output_path)

print(f"Saved processed data to: {output_path}")
print(f"\nData contains:")
print(f"  Cells: {adata.n_obs}")
print(f"  Genes: {adata.n_vars}")
print(f"  HVGs: {adata.var['highly_variable'].sum()}")
print(f"  Layers: {list(adata.layers.keys())}")
print(f"  Embeddings: {list(adata.obsm.keys())}")

## 11. Summary

### Processing Steps Completed
1. Library size normalization (target_sum=10,000)
2. Log transformation
3. HVG selection (n=3,000, seurat_v3 method)
4. Scaling
5. PCA (50 components)
6. Neighborhood graph (15 neighbors)
7. UMAP embedding
8. Leiden clustering (multiple resolutions)

### Data Structure
- `adata.X`: Normalized, log-transformed, scaled expression
- `adata.layers['counts']`: Raw counts
- `adata.layers['normalized']`: Normalized log counts (before scaling)
- `adata.obsm['X_pca']`: PCA embedding
- `adata.obsm['X_umap']`: UMAP embedding

### Next Steps
1. Run doublet detection in `02c_doublet_detection.ipynb`
2. Proceed to integration in `03_integration/`